# CHAPTER 7 - Algorithms for Unconstrained Minimisation

---
---

**Author:** Dr Giordano Scarciotti (g.scarciotti@imperial.ac.uk) - Imperial College London 

**Module:** ELEC70066 - Advanced Optimisation

**Version:** 1.1.3 - 23/02/2023

---
---

The material of this chapter is adapted from $[1]$.

In the next few lectures we will see how convex optimisation problems can be practically solved. In this chapter we start with looking at unconstrained optimisation algorithms, looking back at classics (e.g. Newton method) from a convex optimisation point of view. Contents:


*    Section 7.1 Why can we solve Convex Optimisation Problems in practice?
*    Section 7.2 Background Check
*    Section 7.3 Classical Convergence Analysis of Newton's Method
*    Section 7.4 Self-Concordant Convergence Analysis
*    Section 7.5 Implementation



# 7.1 Why can we solve Convex Optimisation Problems in practice?

In [ ]:
from IPython.display import HTML
HTML('<iframe width="850" height="480" src="https://www.youtube.com/embed/KRGpvbhd6Z4"></iframe>')



We will see that solving convex optimisation problems can be always reduced to the problem of solving a system of linear equations

$$
Ax=b. \tag{1}
$$

Hence, from a numerical point of you, the solution of convex optimisation problems boils down to the use of algorithms for solving the linear equation $(1)$. The topic of efficient solution of equation $(1)$ is outside the scope of this Advanced Optimisation course as it is entirely a linear algebra matter. In this section you will be given a few pointers and ideas on this topic, but essentially the heavy lifting is already done by very solid software packages. In the case of CVX, the software will take care of using the best numerical algorithm for the solution of $(1)$, so you do not really have to worry about any of this. However, this topic may be of importance in your working career if you will end up working on embedded platforms or on very large scale problems for which you have to design your own solver. In this case, a solid knowledge of numerical linear algebra will be essential, so it is important that you are aware of the role of $(1)$ in convex optimisation.

Generally, the most expensive linear algebra operation in terms of computational complexity is the solution of $(1)$ (with products of two matrices being of similar complexity and for which a similar discussion can be made). For a generic matrix $A$, the solution algorithm for $(1)$ will take a number of (floating point) operations which is proportional to $n^3$. However, if $A$ has a special structure, then there exist dedicated algorithms that, by exploiting this structure, can decrease the complexity to $n^2$ and sometimes even to $n$.

For instance, diagonal matrices, triangular matrices, orthogonal matrices and permutation matrices allow trivially $n^2$ or $n$ complexity.

Of course, it is rare to naturally end up with a diagonal matrix. However, another important structure that can be exploited, and it is far more common, is *sparsity*. A sparse matrix is a matrix in which the number of zero elements is much higher than the number of non-zero elements. Sparsity can exploited during the **factor-solve method**. In the factor-solve method, the matrix $A$ is factorised (*factor step*) in a product of matrices with special structure (e.g. triangular) as $A=A_1 A_2 \cdots A_k$. Then the equation $Ax=b$ is solved (*solve step*) as $A_1 x_1 = b$, $A_2 x_2 = x_1$, ..., $A_k x = x_{k-1}$. The point here is that the matrices $A_j$ have structure. For instance they are triangular or permutation matrices. It follows that the solve step is fast, of order $n^2$ or $n$. However, the factor step in general is of order $n^3$. So there is no advantage in using the factor-solve method for general matrices. However, if $A$ is sparse, then the factor step will have complexity much less than $n^3$. In summary, sparsity can be exploited to transform, without too much cost, problem $(1)$ into a series of problems which are easy to solve (because the factorised matrices are e.g. triangular).







Common factorisations which exploit sparsity are the
*    **Sparse $LU$ factorisation**. Given a nonsingular matrix $A$, this can be factorized using the $LU$ factorisation as $A=PLU$, where $P$ is a permutation matrix, $L$ is lower triangular and $U$ is upper triangular. For general matrices, the factorisation costs $\frac{2}{3}n^3$, hence it has negligible advantage. However, if $A$ is sparse, then it can be factorised as $A=P_1 LU P_2$. Adding the permutation matrix $P_2$ offers the possibility of obtaining sparse $L$ and $U$. The cost is less than $\frac{2}{3}n^3$. Cons: the matrices $P_1$ and $P_2$ depend on the sparsity patterns **and the values of $A$**, so the matrices $P_1$ and $P_2$ cannot be reused on $A$'s which have the same structure but different values. 
*    **Sparse Cholesky factorisation**. Given a positive definite matrix $A$, this can be factorized as $A=LL^\top$, where $L$ is lower triangular. For general matrices the factorisation costs $\frac{1}{3}n^3$. However, if $A$ is sparse, then it can be factorised as $A=P LL^\top P^\top$. The cost is usually much less than $\frac{1}{3}n^3$. Unlike sparse $LU$, here the matrix $P$ does not depend on the values of $A$ but only on the sparsity pattern of $A$. So $P$ can be recycled for different $A$'s (as long as they have the same sparsity pattern) saving further time. It is also numerically more stable than $LU$ and provides error bounds. It is preferred to $LU$ whenever possible. Cons: it can be applied only to positive definite matrices.
*    **Sparse $LDL^\top$ factorisation**. Given a nonsingular symmetric matrix $A$, this can be factorized as $A=PLDL^\top P^\top$, where $D$ is a diagonal matrix. The factorisation costs $\frac{1}{3}n^3$ if not sparse, less if $A$ is sparse. Faster than $LU$ (and does not require positive definiteness like Cholesky).



Sometimes we need to solve linear equations with the same matrix $A$ (and of course, different $b_j$'s). In this case the factorisation can be stored. Since the factor step is in general of complexity $n^3$, solving multiple equations with the same matrix $A$ does not increase the computational time. For instance if solving equation $Ax=b_1$ takes $30 $milliseconds, then to solve $100$ equations $Ax=b_j$ with the same $A$ still takes $30$ milliseconds (because the $30$ milliseconds comes from $n^3$ which is much larger than $100n^2$ for large $n$).

There are many other strategies that can be used to solve $Ax=b$. As mentioned above, this topic is outside the scope of the course. The important message here is that convex optimisation problems are solvable in practice because, as we will see, they boils down to solving linear equations. In particular, we will see that convex optimisation problems can be transformed into multiple instances of problems with quadratic functions as objective functions. The gradient of these quadratic functions is linear, i.e. $Ax+b$. Thus, by setting the gradient to zero, we obtain $(1)$. So to solve convex optimisation problems we just need to solve many linear equations $(1)$.

Note that, in general, we wouldn't be able to solve convex optimisation problems with more than several thousands variables, because the complexity of solving $(1)$ is, in general, $n^3$. But it turns out that in convex optimisation problems $A$ is often sparse and so many problems can be solved for millions of variables.

If you want to know more about some of the algorithms for solving $(1)$, you can read Appendix C of $[1]$. None of that material is assessed, and you do not need to know anything beyond the content of this handout that explains why solving $(1)$ is important and why this can be done efficiently.

From now on, you can assume that once a convex optimisation problem is reduced to $(1)$, then it can be solved quickly and reliably.

# 7.2 Background Check

In [ ]:
from IPython.display import HTML
HTML('<iframe width="850" height="480" src="https://www.youtube.com/embed/sC_CMzAkOJ8"></iframe>')

It is suggested that at this point you refresh the material of the module "Optimisation". In particular, the following topics will be given for granted: 

*   Descent methods
*   Line Search methods
*   Gradient descent
*   Netwon's method

We will anyway recall some elements of Newton's method and integrate them with new material.

Recall that a search direction $\Delta x$ is a **descent direction** at $x^{(k)}$ if 

$$
\nabla f(x^{(k)})^\top \Delta x^{(k)} <0.
$$

Recall that the gradient method does not work well on badly conditioned problems (remember the [Rosenbrock's banana function](https://en.wikipedia.org/wiki/Rosenbrock_function)?). For instance, if the level sets of the function to be minimised are narrow ellipsoids, one may get very slow convergence. Note that this problem could be fixed by changing the coordinates of the problem. For instance, consider level sets which are short and wide ellipsoids. If one picks new coordinates that stretch the vertical direction and shrink the orizontal direction, then one would end up with a problem with circular level sets and, hence, fast convergence. In theory this seems like an advantage, but the point is that we hardly know how to select good new coordinates. So in practice, this is a drawback of the gradient method. Mathematically this is indicated by the property that **the gradient method is not coordinate invariant**. This means that the convergence of the gradient method does not depend on the gradient itself, but depends on the specific coordinates in which the problem is written. 

It is well known (and we will show it below) that Newton's method is affine invariant (i.e. invariant with respect to a change of coordinates of the type $\tilde{x}=Tx+t$). However, it is difficult to show that the *speed of convergence* of the method is affine invariant in general. It turns out that for certain classes of convex functions, those called "*self-concordant*", it is indeed possible to show that the speed of convergence of Newton's method is quadratic, irrespectively of the coordinates (which is what we would expect). We will explore this in the following sections.




In the rest of the chapter we assume that we have a suitable starting point $x^{(0)} \in \textbf{dom }f $ and that the sublevel set $S=\{x \in \textbf{dom }f : f(x) \le f(x^{(0)})\}$ is closed.

# 7.3 Classical Convergence Analysis of Newton's Method



In the following we recall (and integrate your knowledge of) Newton's method. We then perform a classical convergence analysis and show its drawbacks.

## 7.3.1 Strong Convexity

In [ ]:
from IPython.display import HTML
HTML('<iframe width="850" height="480" src="https://www.youtube.com/embed/Sw6sDECbkgc"></iframe>')

In this section ($7.3$) we assume that the objective function is **strongly convex** on the set $S$ which means that there exists an $m>0$ such that

$$
\nabla^2 f(x) \succcurlyeq m I \tag{1}
$$

for all $x\in S$. Note that strong convexity implies, but it is not implied by, strict convexity (i.e. $\nabla^2 f(x) \succ 0$). Strong convexity has interesting consequences. We know that applying Taylor's theorem we have that for any $x,y \in S$ there exist $z \in [x,y]$ such that

$$
f(y) = f(x) + \nabla f(x)^\top (y-x) + \frac{1}{2}(y-x)^\top \nabla^2 f(z) (y-x)
$$

(exactly). Applying $(1)$ we have the bound

$$
f(y) \ge f(x) + \nabla f(x)^\top (y-x) + \frac{m}{2}||y-x||^2_2 \tag{2}
$$

for all $x,y\in S$. When $m=0$ we recover the basic inequality that characterises convexity. But when $m\ne 0$ (i.e. when the function is strongly convex), then we have a better bound. In fact, this inequality can be used to bound $f(x) - p^*$ in terms of $||\nabla f(x)||_2$, giving us an assessment of the suboptimality of $x$. To this end, note that the right hand side of $(2)$ can be minimized by computing and setting to zero the gradient with respect to $y$, giving the minimizer $\tilde y = x- (1/m)\nabla f(x)$. By plugging in this minimizer, we obtain

$$
f(y) \ge f(x) - \frac{1}{2m}||\nabla f(x)||^2_2.
$$

Since this holds for all $y\in S$, it holds for the optimal value, so

$$
f(x) - p^* \le \frac{1}{2m}||\nabla f(x)||^2_2. \tag{3}
$$

Thus, if we knew $m$, we could use $(3)$ as a stopping criteria in an algorithm. In practice $m$ is unknown and one would simply stop when $||\nabla f(x)||_2$ is "small". This is well known. However, $(3)$ shows that $||\nabla f(x)||_2$ is potentially a meaningless stopping criteria because convergence does actually depend on $m$ (which is unknown) and has to do with the curvature of the function. The result, again which you know already, is that methods like the gradient descent may perform very badly if the curvature of the function is "unlucky".

Note that the maximum eigenvalue of $\nabla^2 f(x)$ is bounded above on $S$ (because $S$ is assumed closed). This implies that there exists a constant $M >0$ such that

$$
\nabla^2 f(x) \preccurlyeq M I.\tag{4}
$$

By repeating the analysis above, we can find also the lower bound 

$$
f(x) - p^* \ge \frac{1}{2M}||\nabla f(x)||^2_2.
$$

## 7.3.2 Classical Convergence Analysis of the Newton's Method

In [ ]:
from IPython.display import HTML
HTML('<iframe width="850" height="480" src="https://www.youtube.com/embed/T3vSxaRERps"></iframe>')

**Errata:** At 17:06 and 17:17 the video says "strong convergence". It should be "strong convexity".


### Recalling Newton's method

Let $\Delta x_{nt} = -\nabla^2 f(x)^{-1}\nabla f(x)$ be the **Newton step** and define the quantity $\lambda(x)=\left(\nabla f(x)^\top \nabla^2 f(x)^{-1}\nabla f(x)\right)^{\frac{1}{2}}$ as the **Newton decrement**. The Newton decrement can also be expressed using the Newton step as

$$
\lambda(x) = \left(\Delta x_{nt}^\top \nabla^2 f(x)\Delta x_{nt}\right)^{\frac{1}{2}}
$$

which shows that $\lambda$ is the norm of the Newton step with respect to the quadratic norm defined by the Hessian, i.e. $||v||_{\nabla^2 f(x)}=\left(v^\top \nabla^2 f(x) v\right)^{1/2}$. Note that positive definiteness of $\nabla^2 f(x)$ implies that 

$$
\nabla f(x)^\top \Delta x_{nt} = - \lambda(x)^2 < 0
$$

unless $\nabla f(x)=0$, i.e. the Newton step is a descent direction. Let $\hat{f}$ be the second-order approximation of $f$ at $x$, namely

$$
\hat{f}(x+v) = f(x) + \nabla f(x)^\top v + \frac{1}{2}v^\top \nabla^2 f(x) v.
$$

Trivial computation shows that the right-hand side is minimised (with respect to $v$) by $\Delta x_{nt}$. Thus, the Newton step $\Delta x_{nt}$ is what should be added to the point $x$ to minimize the second-order approximation of $f$ at $x$. This interpretation gives us some insight into the Newton step. If the function
$f$ is quadratic, then $x + \Delta x_{nt}$ is the exact minimizer of $f$. If the function $f$ is nearly quadratic, intuition suggests that $x + \Delta x_{nt}$ should be a very good estimate of the minimiser of $f$, i.e., $x^*$. Since $f$ is twice differentiable, the quadratic model of $f$ will be very accurate when $x$ is near $x^*$. Note also that trivial computation shows that

$$
\displaystyle f(x) - \inf_y \hat{f}(y) = f(x) - \hat{f}(x + \Delta x_{nt}) = \frac{1}{2} \lambda(x)^2.
$$

Thus, $\frac{\lambda^2}{2}$ provides an estimate of $f(x) - p^*$ and a stopping criteria for Newton's method.

The analysis above is illustrated in the figure below.



<div>
<img src="https://drive.google.com/uc?export=view&id=1GQvFy2E4trCUrpQgJsF1gCQ4pA-M0XiD" width="500"/>
</div>

Figure 7.1. *The function $f$ (shown solid) and its second-order approximation $\hat f$ at $x$ (dashed). The Newton step $\Delta x_{nt}$ is what must be added to $x$ to give the minimizer of $\hat f$. Source: page $484$ of $[1]$.*

Newton's method is summarized below.

$$
\begin{array}{l}
\textbf{given} \text{ a starting point } x^{(0)} \in \textbf{dom }f\text{, tolerance }\varepsilon >0.\\
\textbf{repeat}\\
\quad 1.\textit{ Compute the Newton step and decrement. }\Delta x^{(k)}_{nt} \text{ and } \lambda(x^{(k)})^2\\
\quad 2.\textit{ Stopping criterion. }\textbf{quit} \text{ if }\lambda^2/2 \le \varepsilon\\
\quad 3.\textit{ Line search. }\text{Choose a step size }t^{(k)}\\
\quad 4.\textit{ Update. }x^{(k+1)}= x^{(k)}+t^{(k)}\Delta x^{(k)}_{nt}
\end{array}
$$

As line search method, in this notes we will use **backtracking** (i.e. Armijo–Goldstein) which we recall here for simplicity

$$
\begin{array}{l}
\textbf{given} \text{ a descent direction } \Delta x \text{ for }f \text{ at }x \in \textbf{dom }f,\, \alpha\in(0,0.5),\,\beta\in(0,1),\,t:=1.\\
\textbf{while } f(x+t\Delta x) > f(x) + \alpha t \nabla f(x)^\top \Delta x, \qquad t:=\beta t
\end{array}
$$

One important property of Newton's method is that it is invariant under affine transformations. This fact has deep implications (as recalled at the beginning of the section).

Suppose $T \in\mathbb{R}^{n\times n}$ is nonsingular and define $\bar{f}(y) = f(Ty)$. Then we have $\nabla \bar{f}(y) = T^\top \nabla f(x)$ and $\nabla^2 \bar{f}(y) = T^\top \nabla^2 f(x) T$ where $x = Ty$. Simple computation show that $\Delta y_{nt}= - (T^T \nabla^2 f(x) T)^{-1}(T^T\nabla f(x)) = -T^{-1}\nabla^2 f(x)^{-1}\nabla f(x) = T^{-1} \Delta x_{nt}$.  Hence, the Newton steps of $f$ and $\bar{f}$ are related by the same linear transformation, i.e.

$$
x + \Delta x_{nt}=T(y + \Delta y_{nt}).
$$

In other words, transforming the problem and then applying Newton is equivalent to applying Newton and then transforming the problem.

**Newton's method does not depends on the coordinates of the problem.**

### Convergence analysis

Assume now that $f$ is strongly convex with constant $m$ and that the Hessian of $f$ is Lipschitz continuous on $S$ with constant $L$, i.e.

$$
||\nabla^2 f(x)-\nabla^2 f(y)||_2 \le L ||x-y||_2
$$

for all $x,y\in S$. The coefficient $L$ can be interpreted as a bound on the third derivative of $f$ (for quadratic functions it can be taken to be zero). In general, a small $L$ indicates that $f$ is approximately quadratic. Thus, we expect that $L$ plays a role in the performance of Newton's method.

Without giving the details of the proof, it can be shown the following result: there exist numbers $\eta$ and $\gamma$ with $0\le \eta \le m^2/L$ and $\gamma > 0$ such that

*   If $||\nabla f(x^{(k)})||_2\ge \eta$ then
$$
f(x^{(k+1)}) - f(x^{(k)}) \le -\gamma
$$
*   If $||\nabla f(x^{(k)})||_2 < \eta $ then the backtracking line search will select $t^{(k)}=1$ and 
$$
\frac{L}{2m^2}||\nabla f(x^{(k+1)})||_2 \le \left( \frac{L}{2m^2}||\nabla f(x^{(k)})||_2 \right)^2 \tag{$*$}
$$

Thus, Newton's algorithm has two phases. 

When $||\nabla f(x^{(k)})||_2\ge \eta$ we have the **damped phase**. In this phase we require a line search and the function value descreases at least by $\gamma$. Thus, if the problem is not unbounded, this phase will ends after at most $(f(x^{(0)})-p^*)/\gamma$ iterations.

When $||\nabla f(x^{(k)})||_2 < \eta$, we have the **quadratically convergent phase**. In this phase all iterations use step $t=1$ (this happens always because it can be shown that when we are in this phase the step $t=1$, which is the first we test in backtracking, always satisfies the exit condition - proof at $[p.491,1]$). Moreover, the quadratically convergent condition will be satisfied for all $l>k$ (because $||\nabla f(x^{(k)})||_2 < \eta$ and $\eta\le m^2/L$, so using $(*)$, $||\nabla f(x^{(k+1)})||_2 < \frac{\eta}{2}$ follows). Thus, we have (again exploiting $(*)$ iteratively and $\eta\le m^2/L$)

$$
\frac{L}{2m^2}||\nabla f(x^{(l)})||_2 \le \left( \frac{L}{2m^2}||\nabla f(x^{(k)})||_2 \right)^{2^{l-k}} \le \left(\frac{1}{2}\right)^{2^{l-k}}.
$$

Using this, in conjunction with $(3)$ yields

$$
f(x^{(l)}) - p^* \le \frac{1}{2m}||\nabla f(x^{(l)})||^2_2  \le \frac{2m^3}{L^2} \left(\frac{1}{2}\right)^{2^{l-k+1}}.
$$

This last inequality implies that we must have $f(x^{(l)})-p^*\le \varepsilon$ after no more than

$$
l-k+1 = \log_2 \log_2 \left(\frac{\varepsilon_0}{\varepsilon}\right)
$$

iterations in the quadratically convergent phase, where $\varepsilon_0 =2m^3/L^2$.

In summary, classical convergence analysis shows that  the number of iterations until $f(x^{(k)})-p^*\le \varepsilon$ is bounded above by

$$
\frac{f(x^{(0)})-p^*}{\gamma} +
\log_2 \log_2 \left(\frac{\varepsilon_0}{\varepsilon}\right)\tag{5}
$$

One can show that $\gamma = \alpha\beta \eta^2 \frac{m}{M^2}$ and $\varepsilon_0 = 2\frac{m^3}{L^2}$. From all practical purposes, $\log_2 \log_2 \left(\frac{\varepsilon_0}{\varepsilon}\right)$ grows so slowly that it can be replaced by a small constant number such as $5$ or $6$.



Newton’s method has several strong advantages over the gradient method, the main two being:

*    Convergence of Newton’s method is rapid in general, and quadratic near $x^*$. Once the quadratic convergence phase is reached, at most six or so iterations
are required to produce a solution with very high accuracy.
*    Newton’s method is affine invariant. It is insensitive to the choice of coordinates, or the condition number of the sublevel sets of the objective function.

The main disadvantage of Newton’s method is the cost of forming and storing the Hessian, and the cost of computing the Newton step, which requires solving a set of linear equations.

We will see later, linking back to the linear algebra section above, that in many cases the computation of the Newton step can be computationally fast (when the Hessian has structure).

Of course, you know from the course "Optimisation" that there are also alternatives to Newton's method, such as the quasi-Newton methods, that offer a computational advantage. You have already spent quite a lot of time on algorithms and, for practical purposes, these are already available and often automatically chosen by software libraries (like CVX). Thus, the study of alternative algorithms are outside the scope of this course. Here we focus on the advantages that convexity brings to the analysis of optimisation algorithms and, for this purpose, Netwon's method is enough.

# 7.4 Self-Concordant Convergence Analysis

We now introduce a more modern approach to convergence analysis due to Nesterov and Nemirovski.

## 7.4.1 Self-concordant Functions

In [ ]:
from IPython.display import HTML
HTML('<iframe width="850" height="480" src="https://www.youtube.com/embed/Rsbfc4ZDqxo"></iframe>')

There are two drawbacks in the classical convergence analysis that we have conducted above. The first is that the constants $\gamma$ and $\varepsilon_0$ depend on $m$, $M$ and $L$, none of which is normally known in practice. Thus, while in theory we have $(5)$, we cannot actually evaluate the bound in practice. The other shortcoming is that we know that Newton's method is coordinate invariant and we know that this is quite a big advantage with respect to the gradient method. However, $m$, $M$ and $L$ all depend on the coordinates. So we have this peculiar situation in which we know that the speed of convergence of the method does not depend on the coordinates, but our bound does.

In this section we seek an alternative to $(5)$ which does not rely on the assumptions 

$$
mI\preccurlyeq \nabla^2 f(x) \preccurlyeq MI, \qquad\qquad ||\nabla^2 f(x)-\nabla^2 f(y)||_2 \le L ||x-y||_2
$$

and that is independent of the coordinates. This is achieved by self-concordance, which is a property introduced by Nesterov and Nemirovski to study the convergence of the interior-point methods for convex optimisation (which we will see in the next lectures). Using self-concordance it is possible to obtain a bound on the speed of convergence which does not depend on any unknown constant and that it is coordinate invariant. Moreover, there are many convex functions that are self-concordant (such as the logarithmic barrier functions appearing in the interior-point methods).

Intuitive explanation: The idea of self-concordance comes from the need of making the bound affine invariant. We know that the bound based on $L$ is related to how small the third derivative is. If the third derivative is small, then the function is almost quadratic and Newton's method will converge fast. So at first we would like to ask something like $|f'''(y)| \le 1$. However, this condition is not coordinate independent as if we use the coordinates $y=ax$ we have $(f(ax))''' = a^3 f'''(ax)$ which may fail to be less than $1$ depending on $a$. The idea here is to get rid of $a^3$ by relating the third and second derivative. Knowning that $(f(ax))'' = a^2 f''(ax)$ we ask that $|f'''(x)|/(|f''(x)|)^{3/2} \le 1$ because then we will have $a^3/(a^2)^{3/2} = 1$.

A convex function $f : \mathbb{R} \to \mathbb{R}$ is **self-concordant**
if

$$
|f′′′(x)| \le 2f′′(x)^{3/2} \tag{6}
$$

for all $x \in \textbf{dom }f$. The factor $2$ is there only for aesthetic reasons. The factor is not important and one can show that if a function $f$ satisfies $(6)$, then the function $\tilde f (x) = 4/k^2 f(x)$ satifies a similar inequality with factor $k$ in place of $2$. What really matters is the power $3/2$ as we saw that this is needed to cancel $a^3$. 

---

**Example 7.1:** Linear and quadratic functions are obviously self-concordant. 

**Exercise 7.1:** show that the negative logarithm $f(x)=-\log x$ is self-concordant.

***EDIT THE FILE TO ADD YOUR PROOF HERE***

**Exercise 7.2:** show that the negative entropy plus negative logarithm $f(x)=x\log x -\log x$ is self-concordant.

***EDIT THE FILE TO ADD YOUR PROOF HERE***

---

We now consider functions on $\mathbb{R}^n$. We say a function $f : \mathbb{R}^n \to \mathbb{R}$
is self-concordant if it is self-concordant along every line in its domain, i.e., if the
function $\tilde{f}(t) = f(x + tv)$ is a self-concordant function of $t$ for all $x\in\textbf{dom }f$ and
for all $v$.

Properties:
*   *Scaling* by a factor exceeding one: If $f$ is self-concordant and $a\ge 1$, then $af$ is self-concordant. 
*   Self-concordance is preserved by *addition*: If $f_1$, $f_2$ are self-concordant, then $f_1 + f_2$ is self-concordant.
*   *Affine invariance*: If $f : \mathbb{R}^n \to \mathbb{R}$ is self-concordant, and $A \in \mathbb{R}^{n\times m}$, $b \in \mathbb{R}^n$, then $f(Ax + b)$ is self-concordant.
*   *Composition with logarithm*: if $g$ is convex with domain $\mathbb{R}_{++}$ and 
$$
|g'''(x)|\le 3 \frac{g''(x)}{x}
$$
then $f(x) = -\log(-g(x)) - \log x$ is self-concordant.



---

**Example 7.2:** the following functions are self-concordant
*   $f(x) = -\sum_{i=1}^m \log(b_i - a_i^\top x)$
*   $f(X) = \log\det X$
*   $f(x) =-\log(y^2 -x^\top x)$

---

## 7.4.2 Analysis of Newton’s Method for Self-concordant Functions

In [ ]:
from IPython.display import HTML
HTML('<iframe width="850" height="480" src="https://www.youtube.com/embed/RhGw2IR5Arc"></iframe>')

We now analyse Newton’s method with backtracking line search, when applied to a strictly convex self-concordant function $f$. As before, we assume that a starting
point $x^{(0)}$ is known, and that the sublevel set $S$ is closed.
We also assume that $f$ is bounded below. The analysis is very similar to the classical analysis, except that we use self-concordance as the basic assumption instead of strong convexity and the Lipschitz condition on the Hessian.

Without giving the details of the proof, it can shown the following result: there exist numbers $\eta$ and $\gamma>0$ with $0\le \eta \le 1/4$ such that

*   If $\lambda (x^{(k)}) > \eta$ then
$$
f(x^{(k+1)}) - f(x^{(k)}) \le -\gamma.
$$
*   If $\lambda (x^{(k)}) \le \eta $ then the backtracking line search will select $t^{(k)}=1$ and 
$$
2\lambda (x^{(k+1)}) \le  \left( 2\lambda (x^{(k)})  \right)^2.
$$


Following an analysis similar to the one done abobe, one can show that the number of iterations until $f(x^{(k)})-p^*\le \varepsilon$ is bounded above by

$$
\frac{f(x^{(0)})-p^*}{\gamma} +
\log_2 \log_2 \left(\frac{1}{\varepsilon}\right) \tag{7}
$$

However, differently from above now one can show that $\gamma = \frac{20 - 8\alpha}{\alpha\beta(1-2\alpha)^2}$, i.e. this bound depends only on known parameters (the coefficients of the backtracking line search).

We already knew (or at least expected) that Newton’s method works in general very well for strongly convex objective functions. We have then showed that we can determine the bound $(5)$ on the number of iterations. However, the bound we found depends on unknown quantities $m$, $M$ and $L$. For self-concordant functions we can say more: we have an iteration bound $(7)$ that is completely explicit, and does not depend on any unknown constant. However, it is important to stress that empirical studies suggest that this bound can be tightened considerably. For instance for $\alpha = 0.1$ and $\beta =0.8$, $(7)$ is around $375(f(x^{(0)})-p^*)+6$. But numerical experiments show that in general something like $c(f(x^{(0)})-p^*)+6$ with $c$ small (like $1.5$) predicts fairly well the number of Newton steps required.

In summary, it is important to stress that self-concordant functions are not necessarily more easily minimised, nor they offer a very tight  bound. Self-concordant functions are just a class of functions for which we can say a bit more about the complexity of Netwon's method (i.e. we have a bound that does not depend on unknown quantities and it is affine invariant).

A last observation to make is that all these bounds require $p^*$ that, of course, is not known. However, one can estimate a lower bound using duality.

# 7.5 Implementation

In [ ]:
from IPython.display import HTML
HTML('<iframe width="850" height="480" src="https://www.youtube.com/embed/l8sLol-zpoU"></iframe>')

**Errata:** In this Chapter it is assumed that the objective function is strictly convex. So when at 1:03 the video says "positive semidefinite", it should be "positive definite".

We now determine the computational complexity of applying Newton's method. In most cases, the work of computing the Newton step dominates the work involved in the line search. 

To compute the Newton step $\Delta x_{nt}$ we first evaluate the Hessian $H = \nabla^2 f(x)$ and the gradient $g = \nabla f(x)$ at $x$. Then we solve the system of linear equations

$$
H \Delta x_{nt} = -g \tag{8}
$$

noticing that this will give us $\Delta x_{nt} = -H^{-1}g = -\nabla^2 f(x)^{-1}\nabla f(x)$.

Since $H$ is symmetric and positive definite (because strict convexity is assumed in this Chapter), we can use the factor-solve method with the Cholesky factorisation. Thus, we find a lower triangular matrix $L$ such that $H = LL^\top$. We then solve $Lw = -g$ (which is trivial because $L$ is triangular) to obtain $w = -L^{-1}g$. Then we solve $L^\top \Delta x_{nt} = w$ (which is again trivial) to obtain

$$
\Delta x_{nt} = L^{-\top}w = -L^{-\top} L^{-1} g = -H^{-1} g.
$$

We know from the section above that for a dense Hessian the computational complexity is of order $n^3$. However, whenever $H$ has some structure, then the complexity descreases.


**Band structure:** If the Hessian $H$ is banded with bandwidth $k$, i.e., $H_{ij} = 0$ for $|i−j| > k$, then the *banded Cholesky* factorization can be used, as well as banded forward and back
substitutions. The cost of computing the Newton step $\Delta x_{nt} = −H^{-1}g$ is then $F +nk^2$ (assuming $k \ll n$ and $F$ the cost of forming the Hessian), compared to $F +(1/3)n^3$ for a dense factorization and substitution method. Note that a banded Hessian is obtained when $f$ can be expressed as a sum of functions of $k$ consecutive variables. For instance, this is the case for the problem

$$
\begin{array}{ll}
\min & \phi_1(x_1,x_2) + \phi_2(x_2,x_3) + \cdots + \phi_{n-1}(x_{n-1},x_n)
\end{array}
$$

for $\phi_i$ convex and twice differentiable.

**Sparse structure:** In the case of a sparse Hessian $H$ we can use the sparse Cholesky factorisation. This happens when in the objective function $f$ each variable $x_i$ is nonlinearly coupled only with a few other variables. Hence, one can find a permutation matrix $P$ and a lower triangular matrix $L$ such that $H= PLL^\top P^\top$. The complexity of the algorithm depends on the specific sparsity pattern, but it is often of order $n$ (instead of $n^3$). Moreover, the sparsity pattern does not change from one step to the other of the Newton method and so the computation of $P$ can be done only once, saving further time.

**Diagonal plus low rank:** Suppose that the Hessian can be expressed as a diagonal matrix plus one of low rank $p$. This is the case for instance when the objective function is

$$
\displaystyle f(x)=\sum_{i=1}^n \phi_i(x_i) + \phi_0(Ax+b)
$$

where $A\in\mathbb{R}^{p\times n}$. Let $D = \textbf{diag}(\phi_1''(x),\dots, \phi_n''(x))$ and $H_0 = \nabla^2 \phi_0(Ax+b)$. Then

$$
H = D + A^\top H_0 A
$$

One can show that solving a linear system involving a diagonal matrix plus one of rank $p$ requires $2p^2 n$, which is less than $(1/3)n^3$ when $p\ll n$.

In summary, we have seen that solving unconstrained convex optimisation problems in a relatevely naive way (standard Newton's method) boils down to solving linear equations. Hence, the computational complexity of solving general unconstrained convex optimisation problems scales with $n^3$. If the Hessian has structure, this can become $n^2$ or even $n$, allowing the solution of very large-scale problems.

In the next chapters we will see that also constrained convex optimisation problems can be reduced to the problem of solving linear equations. Hence, the discussion above holds also for constrained convex optimisation problems.

# End of CHAPTER 7